In [5]:
from operator import itemgetter
from pathlib import Path
import vectorbt as vbt
from hydra.SuperSim import load_prices

def load_portfolio(name):    
    output_dir = Path.cwd().joinpath('..', 'output')
    filepath = output_dir.joinpath(name)
    return vbt.Portfolio.load(filepath)

vbt.settings.ohlcv["column_names"] = {
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close",
    "volume": "volume",
}

pair='XBTUSD'
start_data = "2017-05-15"
end_data = "2017-05-15 02:00"
interval=1


prices = {}
prices[1] = load_prices(pair, '../data/kraken', start_data, end_data, 1)
prices[5] = load_prices(pair, '../data/kraken', start_data, end_data, 5)
prices[15] = load_prices(pair, '../data/kraken', start_data, end_data, 15)
prices[60] = load_prices(pair, '../data/kraken', start_data, end_data, 60)
prices[720] = load_prices(pair, '../data/kraken', start_data, end_data,720)
prices[1440] = load_prices(pair, '../data/kraken', start_data, end_data, 1440)





In [ ]:
AROON = vbt.IndicatorFactory.from_talib("AROON")
print(help(AROON.run))
portfolio = load_portfolio(f"{pair} {start_data} {end_data} {interval} Aroon 100-101.portfolio")
roos = {}
roos[1] = AROON.run(
    prices[1]["high"], prices[1]["low"], timeperiod=[100]
)
roos[5] = AROON.run(
    prices[5]["high"], prices[5]["low"], timeperiod=[100]
)
roos[15] = AROON.run(
    prices[15]["high"], prices[15]["low"], timeperiod=[100]
)
roos[60] = AROON.run(
    prices[60]["high"], prices[60]["low"], timeperiod=[100]
)
roos[720] = AROON.run(
    prices[720]["high"], prices[720]["low"], timeperiod=[100]
)
roos[1440] = AROON.run(
    prices[1440]["high"], prices[1440]["low"], timeperiod=[100]
)

for key, roo in roos.items():
    roo.aroonosc = roo.aroonup - roo.aroondown


In [3]:

# print(portfolio.pos_mask())
# portfolio[[100]].plot()

In [3]:
def parsedatetime(dt):
    return pd.to_datetime(dt)
    # return datetime.strptime(dt, '%Y-%m-%d %H:%M')

def data_extrapolator(dataset, figure, trace, fields):
    def slicer(start, end):
        start_date = parsedatetime(start)
        end_date = parsedatetime(end)
        delta = end_date - start_date
        delta_m = delta.total_seconds() / 60
        if delta_m < 500:
            interval = 1
        elif delta_m < 500 * 5:
            interval = 5
        elif delta_m < 500 * 15:
            interval = 15
        elif delta_m < 500 * 60:
            interval = 60
        elif delta_m < 500 * 720:
            interval = 720
        else:
            interval = 1440

        fakestart = start_date - timedelta(minutes=interval * 250)
        fakeend = end_date + timedelta(minutes=interval * 250)
        precise_data = dataset[interval].loc[fakestart:fakeend]
        orig_data = dataset[1440]
        the_data = pd.concat([orig_data.query('index < @fakestart or index > @fakeend'), precise_data]).sort_index()
        # f = open('log.txt', "a")
        # f.write(f"[{utils.now()}] {start} {end} {delta} {delta_m} {interval} \n{precise_price} \n {fakestart} {fakeend} \n{the_price.loc[fakestart:fakeend]}\n")
        # f.close()
        with figure.batch_update():
            for key, val in fields.items():
                try:
                    value = getattr(the_data, val)
                except:
                    value = the_data[val]

                setattr(trace, key, value)
                

    return slicer

In [4]:
import plotly.graph_objects as go
import pandas as pd
import hydra.utils as utils
from datetime import datetime, timedelta


candlestick = go.Candlestick(
        x=prices[1440].index,
        open=prices[1440]["open"],
        high=prices[1440]["high"],
        low=prices[1440]["low"],
        close=prices[1440]["close"],
        hoverinfo="all",
    )
# print(portfolio.total_profit())
# print(portfolio.orders.buy.records)
# buy_idx = portfolio.orders.buy.records['idx']
# buy_date = prices[1].index[buy_idx]
# buy_price = portfolio.orders.buy.records['price']
# buy_size = portfolio.orders.buy.records['size']
# buy_text = [f"{size} | {price}" for size, price in zip(buy_size, buy_price)]

# sell_idx = portfolio.orders.sell.records['idx']
# sell_date = prices[1].index[sell_idx]
# sell_price = portfolio.orders.sell.records['price']
# sell_size = portfolio.orders.sell.records['size']
# sell_text = [f"{size} | {price}" for size, price in zip(sell_size, sell_price)]


# buys_trace = go.Scattergl(
#     name="Buys",
#     marker_color="green",
#     x=buy_date,
#     y=buy_price,
#     text=buy_text,
#     mode="markers",
#     marker_line_width=2,
#     marker_size=10,
# )

# sells_trace = go.Scattergl(
#     name="Sells",
#     marker_color="red",
#     x=sell_date,
#     y=sell_price,
#     text=sell_text,
#     mode="markers",
#     marker_line_width=2,
#     marker_size=10,
# )


figure = go.FigureWidget(
    # data=[candlestick, buys_trace, sells_trace], 
    data=[candlestick], 
    layout=go.Layout(
        title=dict(
            text='FLYBABYFLY'
        ),
        barmode='overlay',
        yaxis={'fixedrange': False},
        yaxis2=dict(
            fixedrange=False,
            domain=[0,1],
        )
    )
)
cs = figure.data[0]    



update_candlestick = data_extrapolator(prices, figure, cs, {
    "x": "index",
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close"
})

update_candlestick(start_data,end_data)
def handler(obj, xrange):
    [start, end] = xrange.range
    update_candlestick(start, end)
figure.layout.on_change(handler , 'xaxis')
figure

# up_trace = go.Scattergl(
#         name="Aroon Up",
#         marker_color="green",
#         x=aroonup.index,
#         y=aroonup.values,
#         mode="lines",
#     )
# down_trace = go.Scattergl(
#         name="Aroon Down",
#         marker_color="red",
#         x=aroondown.index,
#         y=aroondown.values,
#         mode="lines",
#     )

# aroon_fig = go.FigureWidget(
#     data=[up_trace, down_trace], 
#     layout=go.Layout(
#         title=dict(
#             text='FLYBABYFLY'
#         ),
#         barmode='overlay',
#         yaxis={'fixedrange': False}
#     )
# )

# up = aroon_fig.data[0]    
# down = aroon_fig.data[1]



# update_up = data_extrapolator(prices, figure, up, {
#     "x": "index",
#     "y": "values",
# })
# update_down = data_extrapolator(prices, figure, down, {
#     "x": "index",
#     "y": "values",
# })

# update_up(start_data,end_data)
# update_down(start_data,end_data)
# def aroonhandler(obj, xrange):
#     [start, end] = xrange.range
#     update_candlestick(start, end)
# figure.layout.on_change(aroonhandler, 'xaxis')
# figure
# aroon_fig


FigureWidget({
    'data': [{'close': array([ 8462.8,  8344.9,  8064. , ..., 27376.2, 28879.9, 28959.2]),
    …